#### 1. Create a Local Spark Session

In [ ]:
#create a spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").appName("kmeans-seed-dataset").getOrCreate()
spark

#### 2. Import libraries

In [9]:
#imports for ML pipeline and training
from pyspark.ml.feature import StandardScaler, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

#### 3. Import Seeds dataset from a .csv file

In [10]:
#read dataset from csv file
dataset = spark.read.csv("seeds_dataset.csv",header=True,inferSchema=True)

In [11]:
dataset.head()

Row(A=15.26, P=14.84, C=0.871, LK=5.763, WK=3.312, A_Coef=2.221, LKG=5.22, target=0)

#### 4. Create a Data pre-processing Pipeline 
#####   4.1. Create a vector from all features columns
#####   4.2. Standardize data
#####   4.3. Pass this data to a KMeans object and create the Pipeline object


In [13]:
vec_assembler = VectorAssembler(inputCols = dataset.columns[:-1], outputCol='features')

scaler = StandardScaler(inputCol=vec_assembler.getOutputCol(), outputCol="scaledFeatures", withStd=True, withMean=False)

kmeans = KMeans(featuresCol=scaler.getOutputCol()).setK(2).setSeed(1)

pipeline = Pipeline(stages=[vec_assembler,scaler,kmeans])

#### 5. Fit and Evaluate the model

##### The ClusteringEvaluator use the Silhoute metric do evaluate the model
  
Silhouette analysis can be used to study the separation distance between the resulting clusters. The silhouette plot displays a measure of how close each point in one cluster is to points in the neighboring clusters and thus provides a way to assess parameters like number of clusters visually. This measure has a range of [-1, 1].

Silhouette coefficients (as these values are referred to as) near +1 indicate that the sample is far away from the neighboring clusters. A value of 0 indicates that the sample is on or very close to the decision boundary between two neighboring clusters and negative values indicate that those samples might have been assigned to the wrong cluster.

In [15]:
model = pipeline.fit(dataset)
predictions = model.transform(dataset)


evaluator = ClusteringEvaluator()

silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))

Silhouette with squared euclidean distance = 0.7098457795960431


##### 6. Get the cluster centroids

In [16]:
centers = model.stages[2].clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

Cluster Centers: 
[ 6.2407035  12.29350122 37.40324608 13.82968554  9.69123508  2.31478858
 12.15051313]
[ 4.44396468 10.48536862 36.54671035 12.05177027  8.0111241   2.5455929
 10.33965102]
